# Get Params on Test Data

In [1]:
import pandas as pd
import numpy as np
from autogluon.multimodal import MultiModalPredictor
from sklearn.utils.class_weight import compute_class_weight
import datetime
import os
import torch

e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_float32_matmul_precision("medium")

In [3]:
# Load train and test datasets
train_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/train_df.csv")
test_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/test_df.csv")

In [4]:
# Compute class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_df["label_text"]), y=train_df["label_text"])
class_weights_list = class_weights.tolist()

In [5]:
# Hyperparameters for each model
hyperparam_1 = {
    "Model": "EfficientNet", "gamma": 2.255197916, "learning rate": 0.000198215, "batch_size": 16, "epoch": 13
}
hyperparam_2 = {
    "Model": "EfficientNet", "gamma": 2.49282, "learning rate": 0.000128404, "batch_size": 128, "epoch": 7
}
hyperparam_3 = {
    "Model": "EfficientNet", "gamma": 2.60701, "learning rate": 0.000942346, "batch_size": 16, "epoch": 8
}
hyperparam_4 = {
    "Model": "ResNet18", "gamma": 2.101006332, "learning rate": 0.000719425, "batch_size": 8, "epoch": 9
}
hyperparam_5 = {
    "Model": "ResNet18", "gamma": 2.45143, "learning rate": 0.000269354, "batch_size": 256, "epoch": 7
}
hyperparam_6 = {
    "Model": "ResNet18", "gamma": 2.10302, "learning rate": 0.000678854, "batch_size": 8, "epoch": 9
}
hyperparam_7 = {
    "Model": "EfficientNetv2", "gamma": 2.105304731, "learning rate": 0.000357808, "batch_size": 16, "epoch": 7
}
hyperparam_8 = {
    "Model": "EfficientNetv2", "gamma": 2.62577, "learning rate": 0.000262012, "batch_size": 64, "epoch": 18
}
hyperparam_9 = {
    "Model": "EfficientNetv2", "gamma": 2.25188, "learning rate": 0.000339079, "batch_size": 128, "epoch": 6
}

In [6]:
# Helper function to create the model path and define hyperparameters
def create_and_train_model(hyperparam, idx):
    model_name = hyperparam['Model']
    gamma = hyperparam['gamma']
    learning_rate = hyperparam['learning rate']
    batch_size = hyperparam['batch_size']
    epochs = hyperparam['epoch']

    # Set model-specific identifier
    if model_name == "EfficientNet":
        model_checkpoint = "efficientnet_b2"
    elif model_name == "ResNet18":
        model_checkpoint = "resnet18"
    elif model_name == "EfficientNetv2":
        model_checkpoint = "tf_efficientnetv2_s.in1k"
    else:
        raise ValueError(f"Unknown model: {model_name}")

    # Set training folder name
    now = datetime.datetime.now()
    timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S")
    model_path = f"../../model/master/train_{model_name}_{idx}_{timestamp_str}"
    os.makedirs(model_path, exist_ok=True)

    # Define hyperparameters
    hyperparameters = {
        "optimization.learning_rate": learning_rate,
        "optimization.max_epochs": int(epochs),
        "env.batch_size": int(batch_size),
        "optimization.loss_function": "focal_loss",
        "optimization.focal_loss.alpha": class_weights_list,
        "optimization.focal_loss.gamma": gamma,
        "optimization.focal_loss.reduction": "sum",
        "model.timm_image.checkpoint_name": model_checkpoint,
        "optimization.optim_type": "adamw",
        "optimization.top_k_average_method": "best",
    }

    # Train model
    predictor = MultiModalPredictor(
        label="label_text",
        path=model_path,
        problem_type="multiclass",
        eval_metric="f1_macro",
    )
    predictor.fit(train_data=train_df, hyperparameters=hyperparameters)

    # Evaluate model on train_df
    train_scores = predictor.evaluate(train_df, metrics=["f1_macro", "f1_micro", "accuracy", "recall_macro", "precision_macro"])
    
    # Evaluate model on test_df
    test_scores = predictor.evaluate(test_df, metrics=["f1_macro", "f1_micro", "accuracy", "recall_macro", "precision_macro"])

    # Print train scores
    print(f"Model {model_name} (hyperparameters {idx}) train scores:")
    for metric, score in train_scores.items():
        print(f"  {metric}: {score}")

    # Print test scores
    print(f"Model {model_name} (hyperparameters {idx}) test scores:")
    for metric, score in test_scores.items():
        print(f"  {metric}: {score}")

In [7]:
# Train and evaluate each model with its respective hyperparameters
create_and_train_model(hyperparam_1, 1)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       111.38 GB / 127.91 GB (87.1%)
Disk Space Avail:   1393.77 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_1_2024-12-30_22-18-48
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:12<00:13, 10.32it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.37721 (best 0.37721), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_1_2024-12-30_22-18-48\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:25<00:00, 10.38it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.61362 (best 0.61362), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_1_2024-12-30_22-18-48\\epoch=0-step=134.ckpt' as top 3


...

Epoch 12: 100%|██████████| 269/269 [00:28<00:00,  9.55it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.86it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 18.55it/s]
Model EfficientNet (hyperparameters 1) train scores:
  f1_macro: 0.9993290633412034
  f1_micro: 0.9988674971687429
  accuracy: 0.9988674971687429
  recall_macro: 0.9994979919678716
  precision_macro: 0.9991619526503248
Model EfficientNet (hyperparameters 1) test scores:
  f1_macro: 0.884675895678717
  f1_micro: 0.8601583113456465
  accuracy: 0.8601583113456465
  recall_macro: 0.8930941039636693
  precision_macro: 0.8787630741398287


In [8]:
create_and_train_model(hyperparam_2, 2)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       110.03 GB / 127.91 GB (86.0%)
Disk Space Avail:   1393.74 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_2_2024-12-30_22-25-46
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:12<00:12, 11.05it/s]                 

Epoch 0, global step 8: 'val_f1_macro' reached 0.22420 (best 0.22420), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_2_2024-12-30_22-25-46\\epoch=0-step=8.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:25<00:00, 10.66it/s]

Epoch 0, global step 16: 'val_f1_macro' reached 0.43901 (best 0.43901), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_2_2024-12-30_22-25-46\\epoch=0-step=16.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:26<00:00, 10.12it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.55it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 16.51it/s]
Model EfficientNet (hyperparameters 2) train scores:
  f1_macro: 0.8794564785000265
  f1_micro: 0.8501321253303133
  accuracy: 0.8501321253303133
  recall_macro: 0.9149967358260773
  precision_macro: 0.8522810365755951
Model EfficientNet (hyperparameters 2) test scores:
  f1_macro: 0.6594713740823798
  f1_micro: 0.683377308707124
  accuracy: 0.683377308707124
  recall_macro: 0.6663304422362394
  precision_macro: 0.6887716450216451


In [9]:
create_and_train_model(hyperparam_3, 3)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       110.18 GB / 127.91 GB (86.1%)
Disk Space Avail:   1393.72 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_3_2024-12-30_22-29-55
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:12<00:12, 10.55it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.48908 (best 0.48908), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_3_2024-12-30_22-29-55\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:26<00:00, 10.09it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.54968 (best 0.54968), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_3_2024-12-30_22-29-55\\epoch=0-step=134.ckpt' as top 3


...

Epoch 7: 100%|██████████| 269/269 [00:27<00:00,  9.83it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.58it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 17.60it/s]
Model EfficientNet (hyperparameters 3) train scores:
  f1_macro: 0.9984310589738629
  f1_micro: 0.9977349943374858
  accuracy: 0.9977349943374858
  recall_macro: 0.9980295330983798
  precision_macro: 0.9988425925925926
Model EfficientNet (hyperparameters 3) test scores:
  f1_macro: 0.9129994797072648
  f1_micro: 0.9023746701846965
  accuracy: 0.9023746701846965
  recall_macro: 0.8992751143113463
  precision_macro: 0.9284803696035581


In [10]:
create_and_train_model(hyperparam_4, 4)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       110.02 GB / 127.91 GB (86.0%)
Disk Space Avail:   1393.69 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_4_2024-12-30_22-34-38
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:04<00:04, 29.75it/s]                 

Epoch 0, global step 134: 'val_f1_macro' reached 0.38448 (best 0.38448), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_4_2024-12-30_22-34-38\\epoch=0-step=134.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:09<00:00, 27.37it/s]

Epoch 0, global step 268: 'val_f1_macro' reached 0.62221 (best 0.62221), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_4_2024-12-30_22-34-38\\epoch=0-step=268.ckpt' as top 3


...

Epoch 8: 100%|██████████| 269/269 [00:10<00:00, 26.65it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 38.45it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 22.47it/s]
Model ResNet18 (hyperparameters 4) train scores:
  f1_macro: 0.9943624300623473
  f1_micro: 0.9913174782936958
  accuracy: 0.9913174782936958
  recall_macro: 0.9947390006424053
  precision_macro: 0.9940005160754275
Model ResNet18 (hyperparameters 4) test scores:
  f1_macro: 0.8880079676576389
  f1_micro: 0.8680738786279684
  accuracy: 0.8680738786279684
  recall_macro: 0.8991442266804586
  precision_macro: 0.8787819700031311


In [11]:
create_and_train_model(hyperparam_5, 5)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.79 GB / 127.91 GB (85.8%)
Disk Space Avail:   1393.65 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_5_2024-12-30_22-37-13
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:03<00:03, 39.20it/s]                 

Epoch 0, global step 4: 'val_f1_macro' reached 0.13291 (best 0.13291), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_5_2024-12-30_22-37-13\\epoch=0-step=4.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:07<00:00, 34.92it/s]

Epoch 0, global step 8: 'val_f1_macro' reached 0.17638 (best 0.17638), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_5_2024-12-30_22-37-13\\epoch=0-step=8.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:08<00:00, 32.76it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 38.30it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 24.00it/s]
Model ResNet18 (hyperparameters 5) train scores:
  f1_macro: 0.5503866838143608
  f1_micro: 0.5285013212533032
  accuracy: 0.5285013212533032
  recall_macro: 0.6999065965043245
  precision_macro: 0.5365350870934883
Model ResNet18 (hyperparameters 5) test scores:
  f1_macro: 0.4482421317722825
  f1_micro: 0.46174142480211083
  accuracy: 0.46174142480211083
  recall_macro: 0.5622070964462268
  precision_macro: 0.4694522971684258


In [12]:
create_and_train_model(hyperparam_6, 6)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.63 GB / 127.91 GB (85.7%)
Disk Space Avail:   1393.60 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_6_2024-12-30_22-39-14
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:04<00:04, 30.72it/s]                 

Epoch 0, global step 134: 'val_f1_macro' reached 0.38905 (best 0.38905), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_6_2024-12-30_22-39-14\\epoch=0-step=134.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:09<00:00, 27.70it/s]

Epoch 0, global step 268: 'val_f1_macro' reached 0.63428 (best 0.63428), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_6_2024-12-30_22-39-14\\epoch=0-step=268.ckpt' as top 3


...

Epoch 8: 100%|██████████| 269/269 [00:09<00:00, 27.10it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 37.15it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 21.09it/s]
Model ResNet18 (hyperparameters 6) train scores:
  f1_macro: 0.9948089610154413
  f1_micro: 0.9924499811249529
  accuracy: 0.9924499811249529
  recall_macro: 0.9962074595118967
  precision_macro: 0.9934341292480008
Model ResNet18 (hyperparameters 6) test scores:
  f1_macro: 0.8943761645448456
  f1_micro: 0.8891820580474934
  accuracy: 0.8891820580474934
  recall_macro: 0.8978710433420578
  precision_macro: 0.8924692562691813


In [13]:
create_and_train_model(hyperparam_7, 7)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.90 GB / 127.91 GB (85.9%)
Disk Space Avail:   1393.56 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_7_2024-12-30_22-41-49
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:15<00:16,  8.40it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.65930 (best 0.65930), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_7_2024-12-30_22-41-49\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:33<00:00,  7.90it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.78700 (best 0.78700), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_7_2024-12-30_22-41-49\\epoch=0-step=134.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:35<00:00,  7.51it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.64it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  9.53it/s]
Model EfficientNetv2 (hyperparameters 7) train scores:
  f1_macro: 0.9985529389038161
  f1_micro: 0.9969799924499811
  accuracy: 0.9969799924499811
  recall_macro: 0.9984729126295391
  precision_macro: 0.9986400438559501
Model EfficientNetv2 (hyperparameters 7) test scores:
  f1_macro: 0.9002003815330234
  f1_micro: 0.8891820580474934
  accuracy: 0.8891820580474934
  recall_macro: 0.8939114964477283
  precision_macro: 0.9101166948993035


In [14]:
create_and_train_model(hyperparam_8, 8)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.72 GB / 127.91 GB (85.8%)
Disk Space Avail:   1393.48 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_8_2024-12-30_22-47-09
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:15<00:15,  8.73it/s]                 

Epoch 0, global step 16: 'val_f1_macro' reached 0.31288 (best 0.31288), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_8_2024-12-30_22-47-09\\epoch=0-step=16.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:32<00:00,  8.17it/s]

Epoch 0, global step 33: 'val_f1_macro' reached 0.56787 (best 0.56787), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_8_2024-12-30_22-47-09\\epoch=0-step=33.ckpt' as top 3


...

Epoch 17: 100%|█████████▉| 268/269 [00:32<00:00,  8.19it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.70it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  9.44it/s]
Model EfficientNetv2 (hyperparameters 8) train scores:
  f1_macro: 0.999638471753412
  f1_micro: 0.9992449981124952
  accuracy: 0.9992449981124952
  recall_macro: 0.999665327978581
  precision_macro: 0.9996124031007753
Model EfficientNetv2 (hyperparameters 8) test scores:
  f1_macro: 0.8905912079329289
  f1_micro: 0.8970976253298153
  accuracy: 0.8970976253298153
  recall_macro: 0.8813185338547659
  precision_macro: 0.9036144262187821


In [15]:
create_and_train_model(hyperparam_9, 9)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       109.69 GB / 127.91 GB (85.8%)
Disk Space Avail:   1393.41 GB / 1863.00 GB (74.8%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_9_2024-12-30_22-58-35
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:15<00:15,  8.71it/s]                 

Epoch 0, global step 8: 'val_f1_macro' reached 0.39475 (best 0.39475), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_9_2024-12-30_22-58-35\\epoch=0-step=8.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:33<00:00,  8.07it/s]

Epoch 0, global step 16: 'val_f1_macro' reached 0.64802 (best 0.64802), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_9_2024-12-30_22-58-35\\epoch=0-step=16.ckpt' as top 3


...

Epoch 5: 100%|██████████| 269/269 [00:35<00:00,  7.61it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.51it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.


Predicting DataLoader 0: 100%|██████████| 12/12 [00:01<00:00, 10.63it/s]
Model EfficientNetv2 (hyperparameters 9) train scores:
  f1_macro: 0.9926517254607311
  f1_micro: 0.9890524726311816
  accuracy: 0.9890524726311816
  recall_macro: 0.9949050280375581
  precision_macro: 0.9904759836230589
Model EfficientNetv2 (hyperparameters 9) test scores:
  f1_macro: 0.8588517353276969
  f1_micro: 0.8443271767810027
  accuracy: 0.8443271767810027
  recall_macro: 0.8739148264510583
  precision_macro: 0.8550406939295828
